In [ ]:
# --- 1. Installation (Run this cell once) ---
# !pip install aiograpi nest_asyncio

# --- 2. Imports and Setup ---
import asyncio
import json
import os

# Load environment variables from .env file if it exists
dotenv_path = os.path.join(os.path.dirname('./'), '.env')
if os.path.exists(dotenv_path):
    print(f"Loading environment variables from {dotenv_path}...")
    from dotenv import load_dotenv
    load_dotenv(dotenv_path)
    USERNAME=os.environ.get('USERNAME')
    PASSNAME=os.environ.get('PASSNAME')
else:
    print(f"No .env file found at {dotenv_path}. Skipping loading environment variables.")
from getpass import getpass  # To securely input password

# Import the main client class from aiograpi
from aiograpi import Client

# Import and apply nest_asyncio to allow running asyncio code easily in Jupyter
import nest_asyncio
nest_asyncio.apply()

print("Imports done, nest_asyncio applied.")

# --- 3. Client Instantiation ---
cl = Client()

print("Client object created.")
# adds a random delay between 1 and 3 seconds after each request
cl.delay_range = [1, 3]

# --- 4. Authentication (Choose ONE method) ---

# IMPORTANT: Avoid hardcoding credentials directly in the notebook.
# Use environment variables or getpass for better security.

# Method A: Login with Username and Password (Use with caution!)
# -------------------------------------------------------------
username = os.environ.get(USERNAME) or input("Enter Instagram Username: ")
password = os.environ.get(PASSNAME) or getpass("Enter Instagram Password: ")

async def login_with_password():
    print(f"Attempting login for user: {username}...")
    try:
        await cl.login(username, password)
        print("Login successful!")
        # Optional: Save session settings to avoid logging in next time
        settings_path = "session_settings.json"
        cl.dump_settings(settings_path)
        print(f"Session settings saved to {settings_path}")
        return True
    except Exception as e:
        print(f"Login failed: {e}")
        return False

# Method B: Login with Session ID (Generally Safer)
# -------------------------------------------------
# You need to get your 'sessionid' cookie value from your browser's
# developer tools after logging into instagram.com
# session_id = os.environ.get("INSTAGRAM_SESSIONID") or input("Paste your Instagram sessionid: ")

async def login_with_sessionid():
    print("Attempting login with sessionid...")
    try:
        cl.sessionid = session_id # Set the session ID directly
        # You might need to trigger a request to validate the session,
        # like fetching your own profile info
        my_info = await cl.user_info_by_username(cl.username) # Or fetch your own ID if known
        print(f"Session ID seems valid. Logged in as {my_info.username}")
        # Optional: Save session settings
        settings_path = "session_settings.json"
        cl.dump_settings(settings_path)
        print(f"Session settings saved to {settings_path}")
        return True
    except Exception as e:
        print(f"Login via sessionid failed or session invalid: {e}")
        return False

# # Run the async login function
# login_successful = asyncio.run(login_with_sessionid())

# Method C: Load Session from Saved Settings
# ------------------------------------------
# This is the preferred method after you've logged in successfully once
# using Method A or B and saved the settings.
settings_path = "session_settings.json"

async def load_session():
    if os.path.exists(settings_path):
        print(f"Attempting to load session from {settings_path}...")
        try:
            cl.load_settings(settings_path)
            # Verify the session is still valid by making a simple request
            # Example: Get self account info
            me = await cl.account_info()
            print(f"Session loaded successfully. Logged in as {me.username}")
            return True
        except Exception as e:
            print(f"Failed to load or validate session from file: {e}")
            print("You might need to login again using Method A or B.")
            return False
    else:
        print(f"Session file '{settings_path}' not found. Please login first using Method A or B.")
        return False

# Run the async load function
login_successful = asyncio.run(load_session())
if not login_successful:
    print("Saved Login failed, attempting to login with username and password.")
    # # Run the async login function
    login_successful = asyncio.run(login_with_password())


Loading environment variables from ./.env...
Imports done, nest_asyncio applied.
Client object created.
Attempting login for user: x23430133...
Login successful!
Session settings saved to session_settings.json
Attempting to load session from session_settings.json...
Session loaded successfully. Logged in as x23430133


In [18]:


# --- 5. Basic API Call Example (Only run if login_successful is True) ---

async def get_user_data(username_to_lookup):
    if not login_successful:
        print("Cannot perform API call, login was not successful.")
        return None

    print(f"\nFetching info for user: {username_to_lookup}...")
    try:
        user_info = await cl.user_info_by_username(username_to_lookup)
        print("Successfully fetched user info!")
        # Print some details (user_info is an object, use .dict() or specific attributes)
        # Use .model_dump() for pydantic v2+ or .dict() for v1
        if hasattr(user_info, 'model_dump'):
             print(json.dumps(user_info.model_dump(), indent=2))
        elif hasattr(user_info, 'dict'):
             print(json.dumps(user_info.dict(), indent=2))
        else:
             print(user_info) # Fallback if methods don't exist

        # Example: Get user's first few medias
        # print(f"\nFetching media for user: {username_to_lookup}...")
        # medias = await cl.user_medias(user_info.pk, amount=5) # Fetch 5 posts
        # print(f"Fetched {len(medias)} medias.")
        # for media in medias:
        #     print(f" - Media ID: {media.pk}, Type: {media.media_type}, Caption: {media.caption_text[:50]}...")

        return user_info

    except Exception as e:
        print(f"An error occurred during API call: {e}")
        return None

# --- 6. Run the Example API Call ---
# Replace 'instagram' with the actual username you want to look up
target_username = 'x23430133'

# Run the async function to get user data
# Because we used nest_asyncio, we can use await directly in the cell if login was successful
if login_successful:
    print("\nRunning API call example...")
    user_data = asyncio.run(get_user_data(target_username))
    if user_data:
        print("\nBoilerplate execution finished successfully.")
    else:
        print("\nBoilerplate execution finished with errors during API call.")
else:
    print("\nSkipping API call example because login failed or was skipped.")

# --- Optional: Logout (clears session) ---
# async def logout():
#     print("\nLogging out...")
#     await cl.logout()
#     print("Logged out.")
#     # Optionally remove settings file
#     if os.path.exists(settings_path):
#         os.remove(settings_path)
#         print(f"Removed {settings_path}")

# asyncio.run(logout())


Running API call example...

Fetching info for user: x23430133...


Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/x23430133/?__a=1&__d=dis) >>> 
'data'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/aiograpi/mixins/user.py", line 327, in user_info
    user = await self.user_info_gql(user_id)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/aiograpi/mixins/user.py", line 242, in user_info_gql
    await self.username_from_user_id_gql(user_id)
  File "/opt/anaconda3/lib/python3.12/site-packages/aiograpi/mixins/user.py", line 104, in username_from_user_id_gql
    return (await self.user_short_gql(user_id)).username
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/aiograpi/mixins/user.py", line 78, in user_short_gql
    data = await self.public_graphql_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/aiograpi/mixins/public.py", line 2

Successfully fetched user info!
An error occurred during API call: Object of type Url is not JSON serializable

Boilerplate execution finished with errors during API call.


In [25]:
import asyncio
from aiograpi import Client
from aiograpi.exceptions import ClientError # Import potential exceptions
from typing import List, Any, Optional

# Assume 'cl' is your authenticated aiograpi.Client instance from the previous boilerplate
# Ensure you have successfully run the login/load session part first!
# Example: cl = Client()
# settings_path = "session_settings.json"
# asyncio.run(load_session()) # Make sure login_successful is True

async def get_reels_tray_stories(client: Client, count: int = 10) -> Optional[List[Any]]:
    """
    Fetches a specified number of items from the Reels Tray (Story Feed).

    The Reels Tray typically contains stories from accounts you follow
    and may include suggested/recommended stories based on Instagram's algorithm.

    Args:
        client: An authenticated aiograpi.Client instance.
        count: The maximum number of story tray items (users/reels) to return.

    Returns:
        A list of story tray item objects (up to 'count') if successful,
        otherwise None if an error occurs or the client isn't logged in.
        The exact type of items in the list depends on the aiograpi library version.
    """
    if not client.user_id:
        print("Error: Client does not appear to be logged in.")
        return None

    print(f"Attempting to fetch the first {count} items from the Reels Tray...")
    try:
        # Call the method to get the story tray data
        # This usually fetches the entire current tray provided by the API
        tray_items: dict[Any] = await client.get_reels_tray_feed()
        tray_items = list(tray_items) # Convert to list if needed
        print("got here")
        if not tray_items:
            print("No items found in the Reels Tray.")
            return []

        # Return the requested number of items by slicing the list
        requested_items = tray_items[:count]
        print(f"Successfully fetched {len(requested_items)} items from the Reels Tray.")
        return requested_items

    except ClientError as e:
        print(f"An error occurred (ClientError): {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# --- How to use the function ---

async def main_story_fetch(client: Client, num_stories: int):
    """Main async function to call the story fetching logic."""
    # Ensure client is logged in before calling (checked inside get_reels_tray_stories too)
    if not client.user_id:
         print("Please ensure you are logged in before fetching stories.")
         return

    story_items = await get_reels_tray_stories(client, count=num_stories)

    if story_items is not None:
        print(f"\n--- Got {len(story_items)} Story Tray Items ---")
        if not story_items:
            print("(The tray was empty or returned no items)")
        else:
            for idx, item in enumerate(story_items):
                # The structure of 'item' might vary. Common attributes include
                # 'user' (with username, pk), 'items' (list of actual story posts)
                # Inspect the 'item' object to see its attributes (e.g., print(dir(item)))
                username = "Unknown User"
                num_story_segments = 0

                if hasattr(item, 'user') and hasattr(item.user, 'username'):
                     username = item.user.username
                elif hasattr(item, 'owner') and hasattr(item.owner, 'username'): # Alternative structure check
                     username = item.owner.username


                if hasattr(item, 'items') and isinstance(item.items, list):
                    num_story_segments = len(item.items)
                elif hasattr(item, 'media_ids') and isinstance(item.media_ids, list): # Alternative structure check
                    num_story_segments = len(item.media_ids)


                print(f"{idx + 1}. User: {username} ({num_story_segments} segments)")
                # You can further process each 'item' here, e.g., fetch individual story media
                # print(item) # Uncomment to see the full structure of an item

# --- Example Execution ---
# Make sure 'cl' is your authenticated client object from the previous setup

# Set the number of story tray items you want
desired_story_count = 5

# Run the main async function using asyncio.run() or await if in a running loop/notebook
if 'cl' in locals() and cl.user_id: # Check if cl exists and seems logged in
    print("\nRunning the story fetching example...")
    # print(dir(cl)) # Optional: Check available methods and attributes of the client
    asyncio.run(main_story_fetch(cl, desired_story_count))
    print("\nStory fetching example finished.")
else:
    print("\nClient 'cl' not found or not logged in. Skipping story fetching example.")
    print("Please run the authentication steps from the boilerplate first.")


Running the story fetching example...
Attempting to fetch the first 5 items from the Reels Tray...
got here
Successfully fetched 5 items from the Reels Tray.

--- Got 5 Story Tray Items ---
1. User: Unknown User (0 segments)
2. User: Unknown User (0 segments)
3. User: Unknown User (0 segments)
4. User: Unknown User (0 segments)
5. User: Unknown User (0 segments)

Story fetching example finished.
